In [ ]:
import os

local_path = os.getcwd()
# 设置工作目录为项目的主目录
os.chdir("../../")  # 使用相对路径将工作目录切换到 project 文件夹
print("Current working directory:", os.getcwd())

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

from models import modelset
from train.train import train_FBM
from utils import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
set_seed(42)

# 生成数据

两类数据为在高维空间中球形分布的GAUSS数据。如果两个球存在交叠，处于交叠区域的数据将会被标记为第三类。

In [ ]:
import torch

def generate_sphere_data_with_overlap(
    num_samples_per_class, dimensions, radius_0, radius_1, center, mix_rate
):
    """
    生成两个类别的数据集，每类数据分布在一个高维球体内，这些数据具有相同球心。在不同半径下属于不同的类别。
    存在混合半径，如果在该半径下，无法区分两类数据，用另一个标签标记。
    
    :param num_samples_per_class: 每个类别的样本数量
    :param dimensions: 高维空间的维数
    :param radius_0:    类别 0 球体的半径，最长的半径
    :param radius_1:    类别 1 球体的半径
    :param center:      球体的球心位置 (list or tensor)
    :param mix_rate:    混合半径占占两个半径的差别的比例
    :return: 数据集 (features, labels)
    """
    def generate_points(num_samples, dimensions, radius_0, radius_1, radius_mix, center):
        # radius_0 是最长的半径，radius_1是次长的半径
        data = []
        labels = []
        center_tensor_0 = torch.tensor(center, dtype=torch.float32)
        
        while len(data) < num_samples:
            # 从均匀分布生成点
            point = (torch.rand(dimensions)-0.5) * 2 * radius_0/math.sqrt(dimensions) 
            # 计算点到球心的距离
            distance_to_center = torch.norm(point)
            point += center_tensor_0
            
            # 检查点是否在类别 0 和类别 1 的球体内
            in_sphere_0 = distance_to_center <= radius_0
            in_sphere_1 = distance_to_center <= radius_1
            in_sphere_mix = distance_to_center <= radius_mix
            
            if in_sphere_1:         #是否在范围内
                data.append(point)
                labels.append(1)
            elif in_sphere_mix:
                data.append(point)
                labels.append(2)
            elif in_sphere_0:
                data.append(point)
                labels.append(0)

        return torch.stack(data), torch.tensor(labels, dtype=torch.long)
    
    radius_mix = (radius_0 - radius_1) * mix_rate + radius_1
    # 生成数据和标签
    features, labels = generate_points(
        num_samples_per_class,  # 生成更多点以处理两类重叠的情况
        dimensions,
        radius_0,
        radius_1,
        radius_mix,
        center
    )


    #features, labels = torch.cat((features), dim=0), torch.cat((labels), dim=0)

    return features, labels



# 可视化
通过PCA（TSNE）的方式将数据降为成2维度数据画图

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 可视化函数：将数据降维到 2D 并可视化
def visualize_2d(features, labels):
    """
    使用 t-SNE 将高维数据降维到 2D 并可视化。
    :param features: 高维特征数据 (tensor or numpy array)
    :param labels: 标签 (tensor or numpy array)
    """
    # 转换为 numpy
    features_np = features.detach().numpy()
    labels_np = labels.numpy()

    # 使用 t-SNE 降维到 2D
    _, dims =features.shape
    
    if dims > 2:
        #tsne = TSNE(n_components=2, random_state=42, perplexity=40)
        pca = PCA(n_components=2)
        features_2d = pca.fit_transform(features_np)
        explained_variance_ratio = pca.explained_variance_ratio_
    else:
        features_2d = features_np
        explained_variance_ratio = None

    # 绘制 2D 散点图
    plt.figure(figsize=(8, 6))
    for label in [0, 1, 2]:
        mask = labels_np == label
        plt.scatter(
            features_2d[mask, 0],
            features_2d[mask, 1],
            label=f"Class {label}",
            alpha=0.6
        )
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.title("Visualization of High-Dimensional Data")
    plt.legend()
    plt.grid(True)

    # 显示 PCA 信息占比
    if explained_variance_ratio is not None:
        info_text = f"Explained Variance:\nDim 1: {explained_variance_ratio[0]:.2%}\nDim 2: {explained_variance_ratio[1]:.2%}"
        plt.text(0.05, 0.95, info_text, transform=plt.gca().transAxes, fontsize=10,
                 verticalalignment='top', bbox=dict(boxstyle='round', alpha=0.2))

    plt.show()

In [ ]:
# 示例：生成每类 500 个样本的高维数据
num_samples = 2000
dimensions = 2
radius_0 = 2.0
radius_1 = 1.0
mix_rate = 0.0

center = [0.0] * dimensions  # 球心位置


# 生成数据
features, labels = generate_sphere_data_with_overlap(
    num_samples, dimensions, radius_0, radius_1, center, mix_rate
)

# 使用可视化
visualize_2d(features, labels)

# 数据转换
将数据集转化为 TensorDataset，方便接下来的训练

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 找到标签为 2 的索引
idx_2 = (labels == 2)
num_idx_2 = idx_2.sum().item()
# 随机生成 0 或 1 的替换标签
random_labels = torch.randint(0, 2, (num_idx_2,), dtype=torch.long)
# 替换标签
labels[idx_2] = random_labels

# 将 features 和 labels 转换为 TensorDataset
dataset = TensorDataset(features, labels)

# 使用 DataLoader 以便进行批量加载
batch_size = 64  # 设置每个 batch 的大小
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 检查 DataLoader 输出
for batch_features, batch_labels in train_loader:
    print("Batch features shape:", batch_features.shape)
    print("Batch labels shape:", batch_labels.shape)
    break


# 利用构造的数据集对FBM进行训练

In [ ]:
from loss.loss import FBMLoss
from models.modelset import FBMLayer
import torch.optim as optim

# 定义超参数
hidden_dim =10
num_classes = 2      # MNIST有10个类别
learning_rate = 0.01
num_epochs = 20
batch_size = 64
alpha = 1.0
df = 0.02

# 实例化模型、定义损失函数和优化器
model = FBMLayer(dimensions, hidden_dim).to(device)
criterion = FBMLoss(hidden_dim, 0.01, df, alpha)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()
# 训练模型
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # 将图像和标签移动到 GPU 上
        images = images.view(-1, dimensions).to(device)  # 展平图像并转移到 GPU
        labels = labels.to(device)  # 标签移动到 GPU
        #labels_one_hot = F.one_hot(labels, num_classes=num_classes).float()
        
        # 前向传播
        outputs = model(images)
        #loss = criterion(outputs, labels_one_hot, model.linear.weight)
        loss = criterion(outputs, labels, model.linear.weight)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# 示例：生成每类 500 个样本的高维数据
num_samples = 500

# 生成数据
features, labels = generate_sphere_data_with_overlap(
    num_samples, dimensions, radius_0, radius_1, center, mix_rate
)


# 使用可视化
visualize_2d(features, labels)

In [ ]:
visualize_2d(model(features.to(device)).cpu(), labels)

In [ ]:
model.linear.weight
model(features.to(device))[340]